# PROJECT ON QUERY OPTIMIZATION 
#          By: Girma Moges and Temesgen Muche

In [4]:
# !pip install pymysql
# !pip install numpy

In [6]:
import pymysql
import random
import numpy as np

connection = pymysql.connect(host='localhost', user='root', password='Girma@1223', db='world')
cursor = connection.cursor()

In [7]:
# Execute the SQL query
sql_query = "SELECT * FROM country WHERE continent = 'Asia' AND IndepYear >= 103000 ORDER BY IndepYear DESC LIMIT 100;"
cursor.execute(sql_query)
result = cursor.fetchall()

# Print the results
for row in result:
    print(row)

In [8]:
def evaluate(query):
    start_time = time.time()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time

In [9]:
POPULATION_SIZE = 20
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8
ELITE_SIZE = 2
GENERATIONS = 50

In [10]:
class Chromosome:
    def __init__(self, genes):
        self.genes = genes
        self.fitness = 0
        
    def __repr__(self):
        return f'Chromosome({self.genes}, {self.fitness})'

In [11]:
def evaluate_fitness(chromosome):
    sql_query = f"SELECT * FROM country WHERE {chromosome.genes} ORDER BY IndepYear DESC LIMIT 100;"
    cursor.execute(sql_query)
    result = cursor.fetchall()
    chromosome.fitness = len(result)
    return chromosome.fitness

In [12]:
def mutation(chromosome):
    genes = chromosome.genes.split(' AND ')
    for i in range(len(genes)):
        if random.random() < MUTATION_RATE:
            gene = genes[i]
            if '=' in gene:
                column, value = gene.split('=')
                if column.strip() == 'continent':
                    value = "'" + random.choice(['Asia', 'North America', 'Africa', 'South America', 'Oceania', 'Antarctica', 'Europe ']) + "'"
                elif column.strip() == 'IndepYear':
                    value = "'" + f'{np.random.randint(1945, 1994)}-{np.random.randint(1, 13):02d}-{np.random.randint(1, 29):02d}' + "'"
                else:
                    value = np.random.randint(1, 50001)
                genes[i] = f'{column.strip()}={value}'
            elif '>' in gene:
                column, value = gene.split('>')
                value = np.random.randint(int(value)+1, 50001)
                genes[i] = f'{column.strip()}>{value}'
            elif '<' in gene:
                column, value = gene.split('<')
                value = np.random.randint(0, int(value)-1)
                genes[i] = f'{column.strip()}<{value}'
    return Chromosome(' AND '.join(genes))

def crossover(parent1, parent2):
    genes1 = parent1.genes.split(' AND ')
    genes2 = parent2.genes.split(' AND ')
    if random.random() < CROSSOVER_RATE:
        index1 = np.random.randint(len(genes1))
        index2 = np.random.randint(len(genes2))
        genes1[index1:], genes2[index2:] = genes2[index2:], genes1[index1:]
    return Chromosome(' AND '.join(genes1)), Chromosome(' AND '.join(genes2))

In [13]:
def generate_population(size):
    population = []
    for i in range(size):
        genes = []
        genes.append(f"continent='{random.choice(['Asia', 'North America', 'Africa', 'South America', 'Oceania', 'Antarctica', 'Europe '])}'")
        genes.append(f"IndepYear >='{np.random.randint(1945, 1994)}-{np.random.randint(1, 13):02d}-{np.random.randint(1, 29):02d}'")
#         genes.append(f"ID>{np.random.randint(1, 50)}")
        chromosome = Chromosome(' AND '.join(genes))
        population.append(chromosome)
    return population

population = generate_population(POPULATION_SIZE)

In [14]:
def select_best_chromosomes(population, elite_size):
    sorted_population = sorted(population, key=lambda chromosome: chromosome.fitness, reverse=True)
    return sorted_population[:elite_size] + random.sample(sorted_population[elite_size:], len(population) - elite_size)

In [15]:
best_fitness = 0
for i in range(GENERATIONS):
    new_population = []
    elites = select_best_chromosomes(population, ELITE_SIZE)
    new_population += elites
    
    while len(new_population) < POPULATION_SIZE:
        parent1, parent2 = random.sample(elites, 2)
        child1, child2 = crossover(parent1, parent2)
        child1 = mutation(child1)
        child2 = mutation(child2)
        new_population.append(child1)
        new_population.append(child2)
    
    for chromosome in new_population:
        evaluate_fitness(chromosome)
        if chromosome.fitness > best_fitness:
            best_fitness = chromosome.fitness
            best_chromosome = chromosome
            
    population = new_population
print(f'Generation {i+1}, Best Fitness: {best_fitness}, Best Chromosome: {best_chromosome}')

Generation 50, Best Fitness: 11, Best Chromosome: Chromosome(continent='North America' AND IndepYear >='1960-07-20', 11)


In [16]:
import random

# Define the chromosome class
class Chromosome:
    def __init__(self, continent, population, emp_no, fitness):
        self.continent = continent
        self.population = population
        self.emp_no = emp_no
        self.fitness = fitness

# Define the fitness function
def calculate_fitness(chromosome):
    # Here, you would define your fitness function to evaluate the fitness of the chromosome
    # and return the fitness value as an integer
    return random.randint(500, 1500)

# Define the genetic algorithm
def genetic_algorithm():
    # Initialize the population with random chromosomes
    population = [Chromosome(continent='Asia', population='10300', emp_no=random.randint(1000, 50000), fitness=0) for i in range(100)]
    
    # Iterate over a number of generations
    for generation in range(50):
    
        # Evaluate the fitness of each chromosome in the population
        for chromosome in population:
            chromosome.fitness = calculate_fitness(chromosome)
    
        # Sort the population by fitness, in descending order
        population.sort(key=lambda x: x.fitness, reverse=True)
    
        # Print the best chromosome and its fitness for this generation
        print(f"Generation {generation+1}, Best Fitness: {population[0].fitness}, Best Chromosome: {population[0]}")
    
        # Select the top 10% of the population as parents for the next generation
        num_parents = int(len(population) * 0.1)
        parents = population[:num_parents]
        
        # Crossover the parents to create offspring
        offspring = []
        for i in range(len(population) - num_parents):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            child = Chromosome(continent=parent1.continent, population=parent2.population,
                               emp_no=random.randint(1000, 50000), fitness=0)
            offspring.append(child)
        
        # Mutate the offspring
            for child in offspring:
                if random.random() < 0.1:
                    child.population = str(int(child.population) + random.randint(-1000, 1000))

            # Create the next generation by combining the parents and offspring
            population = parents + offspring

        # Return the best chromosome from the final generation
        return max(population, key=lambda x: x.fitness)

# Run the genetic algorithm
best_chromosome = genetic_algorithm()
print(f"The best chromosome found: {best_chromosome}")

Generation 1, Best Fitness: 1479, Best Chromosome: <__main__.Chromosome object at 0x0000027745397C40>
The best chromosome found: <__main__.Chromosome object at 0x0000027745397C40>


In [17]:
import time
import pymysql
import random
import numpy as np

connection = pymysql.connect(host='localhost', user='root', password='Girma@1223', db='world')
cursor = connection.cursor()

# Define the unoptimized query
unoptimized_query = "SELECT * FROM country WHERE continent = 'Asia' AND IndepYear >= '1960-07-20' ORDER BY IndepYear DESC LIMIT 100;"

# Define the optimized query
optimized_query = "SELECT * FROM country WHERE continent = 'Asia' AND IndepYear >= '1960-07-20' ORDER BY IndepYear DESC LIMIT 100;"

class Chromosome:
    def __init__(self, query, fitness):
        self.query = query
        self.fitness = fitness
        
    def __repr__(self):
        return f'Chromosome({self.query}, {self.fitness})'

def evaluate(query):
    start_time = time.time()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time

POPULATION_SIZE = 20
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8
ELITE_SIZE = 2
GENERATIONS = 50

def evaluate_fitness(chromosome):
    execution_time = evaluate(chromosome.query)
    if execution_time == 0:
        chromosome.fitness = 0
    else:
        chromosome.fitness = 1 / execution_time
    return chromosome.fitness
def mutation(chromosome):
    if random.random() < MUTATION_RATE:
        if chromosome.query == unoptimized_query:
            return Chromosome(optimized_query, 0)
        else:
            return Chromosome(unoptimized_query, 0)
    else:
        return chromosome

def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        if parent1.query == unoptimized_query and parent2.query == unoptimized_query:
            child_query = unoptimized_query
        elif parent1.query == optimized_query and parent2.query == optimized_query:
            child_query = optimized_query
        else:
            child_query = random.choice([parent1.query, parent2.query])
        return Chromosome(child_query, 0), Chromosome(child_query, 0)
    else:
        return parent1, parent2
def generate_population(size):
    population = []
    for i in range(size):
        query = random.choice([unoptimized_query, optimized_query])
        chromosome = Chromosome(query, 0)
        population.append(chromosome)
    return population
def select_best_chromosomes(population, elite_size):
    sorted_population = sorted(population, key=lambda chromosome: chromosome.fitness, reverse=True)
    return sorted_population[:elite_size]

population = generate_population(POPULATION_SIZE)

best_fitness = 0
for i in range(GENERATIONS):
    new_population = []
    elites = select_best_chromosomes(population, ELITE_SIZE)
    new_population += elites
    
    while len(new_population) < POPULATION_SIZE:
        parent1, parent2 = random.sample(elites, 2)
        child1, child2 = crossover(parent1, parent2)
        child1 = mutation(child1)
        child2 = mutation(child2)
        new_population.append(child1)
        new_population.append(child2)
        
    for chromosome in new_population:
        evaluate_fitness(chromosome)
        if chromosome.fitness > best_fitness:
            best_fitness = chromosome.fitness
            best_chromosome = chromosome
            
    population = new_population

print(f'Best Chromosome: {best_chromosome}')

Best Chromosome: Chromosome(SELECT * FROM country WHERE continent = 'Asia' AND IndepYear >= '1960-07-20' ORDER BY IndepYear DESC LIMIT 100;, 333.19860184302513)
